In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import benchlingapi
from itertools import product
from cytoolz import reduce

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.design as design

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Config

In [ ]:
plib = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]

# Import plasmids

# Choose cutters

## Load sequences

In [ ]:
# TODO: avoid cutting in the interesting part of our sequences (e.g., parts+GG sites), if possible

In [ ]:
plib.remote["Tags"].value_counts()

In [ ]:
plib_plasmids = reg.get_df(("pLIB", "plasmids"))

In [ ]:
plib_plasmids

In [ ]:
plib_plasmids["Tags"].value_counts()

In [ ]:
plib_plasmids[plib_plasmids["Tags"].str.contains("sequestration")]

In [ ]:
# plasmids = {f"pLIB{num}": reg.get(f"pLIB{num}")["_seq"] for num in range(219, 223)}

### Temp

In [ ]:
import paulssonlab.cloning.io as io
from glob import glob
import os
import re

In [ ]:
plasmids = {
    re.search(r"\((\w+)\)", f).group(1): io.read_file(f)
    for f in glob(os.path.expanduser("~/Downloads/benchling_export/*.gb"))
}

In [ ]:
plasmids

## Find single cutters

In [ ]:
single_cutters = {
    name: {
        e
        for e, cuts in Restriction.CommOnly.search(plasmid.seq).items()
        if len(cuts) == 1
    }
    for name, plasmid in plasmids.items()
}

In [ ]:
reduce(set.intersection, single_cutters.values())

In [ ]:
single_cutters["pSKA417"]

In [ ]:
single_cutters["pSKA417"] & single_cutters["pSKA539"]

In [ ]:
single_cutters["pSKA417"] & single_cutters["pSKA539"] & single_cutters["pSKA562"]